# Model Processing

Create three variables and mask them correctly for comparison with observations.

1. Total Cloud Amount from MISR

2. Thick-Low Cloud Amount from MISR

3. Thick-High Cloud Amount from MODIS

I will store them in the new 'taylor_files' directory for consistency (I don't have write access to all of the post-processed data).

### I am just doing the 2012 files and the CAM6 run right now. Easy enough to come back when the other runs are finalized.

Import packages, functions, and classes

In [1]:
import sys
# Add common resources folder to path
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis')
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis/Common/')
# sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable, cmaps
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png,calculate,dual_mask
    )

from cloud_metric import Cloud_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the post-processed files using a CloudMetric class (just makes it easier)

In [2]:
case_dir = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/'

workmetric = Cloud_Metric(case_dir)
workmetric.add_case('cam4_1deg_release_amip',label='CAM4')
workmetric.add_case('cam5_1deg_release_amip',label='CAM5')
workmetric.add_case(case='f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1',
                    path='/glade/p/cesm/pcwg/jenkay/COSP/cesm21/',label='CAM6')

# fig2metric = Cloud_Metric(case_dir)
# fig2metric.add_case('cam4_1deg_release_amip',label='CAM4')
# fig2metric.add_case('cam5_1deg_release_amip',label='CAM5')
# fig2metric.add_case(case='f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1',path='/glade/p/cesm/pcwg/jenkay/COSP/cesm21/',label='CAM6 CMIP6 AMIP')


Loading GOCCP data...done.
Loading CALIOP SLFs...Could not load cloudtop CALIOP slfs from caliop_olimpia/ct_slf_olimpia/cloudtop_slfs.nc
Loading CERES-EBAF fluxes...done.
Loading ISCCP cloud total...done.
Loading MISR cloud histograms...done.
Loading MODIS cloud histograms...done.
Processed timeseries directory found for cam4_1deg_release_amip. Will load data as required.
cam4_1deg_release_amip load successfully.
Processed timeseries directory found for cam5_1deg_release_amip. Will load data as required.
cam5_1deg_release_amip load successfully.
Processed timeseries directory found for f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1. Will load data as required.
f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1 load successfully.


### Load the standard variables I will be working with

In [3]:
workmetric.load_vars(['CLD_MISR','CLMODIS']);#,'CLDTOT_ISCCP','CLDTOT_CAL']);

In [4]:
cam6_da = workmetric.get_case('f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1').case_da
cam5_da = workmetric.get_case('cam5_1deg_release_amip').case_da
cam4_da = workmetric.get_case('cam4_1deg_release_amip').case_da

#### Manually create the total MISR cloud file

This is slow because operations over the 5-d histogram arrays take a long time.

#### CAM6

In [10]:
cldtot_misr = cam6_da[['CLD_MISR','time_bnds']].where(cam6_da['CLD_MISR'].cosp_tau>0.3).sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam6_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldtot_misr_msk = cldtot_misr.where(mask > -100)

cldtot_misr_msk['time_bnds'] = cam6_da['time_bnds'] # add time bounds so it can load normally

cldtot_misr_msk = cldtot_misr_msk.rename({'CLD_MISR':'CLDTOT_MISR'}) # rename appropriately

In [24]:
cam6_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam6_runname = 'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1'

# cldtot_misr_msk.to_netcdf('%s/%s/%s.CLDTOT_MISR.nc' % (cam6_tempdir,cam6_runname,cam6_runname))

#### CAM5

In [31]:
cldtot_misr = cam5_da[['CLD_MISR','time_bnds']].where(cam5_da['CLD_MISR'].cosp_tau>0.3).sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam5_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nan, easy

cldtot_misr_msk = cldtot_misr.where(mask > -100)

cldtot_misr_msk['time_bnds'] = cam5_da['time_bnds'] # add time bounds so it can load normally

cldtot_misr_msk = cldtot_misr_msk.rename({'CLD_MISR':'CLDTOT_MISR'}) # rename appropriately

In [32]:
cam5_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam5_runname = 'cam5_1deg_release_amip'

# cldtot_misr_msk.to_netcdf('%s/%s/%s.CLDTOT_MISR.nc' % (cam5_tempdir,cam5_runname,cam5_runname))

#### CAM4

In [35]:
cldtot_misr = cam4_da[['CLD_MISR','time_bnds']].where(cam4_da['CLD_MISR'].cosp_tau>0.3).sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam4_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nan, easy

cldtot_misr_msk = cldtot_misr.where(mask > -100)

cldtot_misr_msk['time_bnds'] = cam4_da['time_bnds'] # add time bounds so it can load normally

cldtot_misr_msk = cldtot_misr_msk.rename({'CLD_MISR':'CLDTOT_MISR'}) # rename appropriately

KeyboardInterrupt: 

In [34]:
cam4_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam4_runname = 'cam4_1deg_release_amip'

# cldtot_misr_msk.to_netcdf('%s/%s/%s.CLDTOT_MISR.nc' % (cam4_tempdir,cam4_runname,cam4_runname))

#### Manually create the thick-low MISR cloud file

__Note that the height units differ between COSP1.3 and COSP2, so indexing works better than using a where statement (and is also far faster)!__

#### CAM6

In [19]:
cldthick_misr = cam6_da['CLD_MISR'][:,1:7,-2:,:,:].sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam6_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_misr_msk = cldthick_misr.where(mask > -100)

cam6_da['CLDTHCK_MISR'] = cldthick_misr_msk # add the new thick cloud variable back in
cldthick_misr_msk = cam6_da[['CLDTHCK_MISR','time_bnds']] # select the new variable and the time_bnds variable

In [20]:
cam6_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam6_runname = 'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1'

cldthick_misr_msk.to_netcdf('%s/%s/%s.CLDTHCK_MISR.nc' % (cam6_tempdir,cam6_runname,cam6_runname))

#### CAM5

For both COSP versions, the first height index corresponds to a non-retrieval and should be excluded.

In [22]:
cldthick_misr = cam5_da['CLD_MISR'][:,1:7,-2:,:,:].sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam5_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_misr_msk = cldthick_misr.where(mask > -100)

cam5_da['CLDTHCK_MISR'] = cldthick_misr_msk # add the new thick cloud variable back in
cldthick_misr_msk = cam5_da[['CLDTHCK_MISR','time_bnds']] # select the new variable and the time_bnds variable

In [27]:
cam5_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam5_runname = 'cam5_1deg_release_amip'

cldthick_misr_msk.to_netcdf('%s/%s/%s.CLDTHCK_MISR.nc' % (cam5_tempdir,cam5_runname,cam5_runname))

#### CAM4

In [29]:
cldthick_misr = cam4_da['CLD_MISR'][:,1:7,-2:,:,:].sum(dim=['cosp_tau','cosp_htmisr'],skipna=True)

mask = cam4_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_misr_msk = cldthick_misr.where(mask > -100)

cam4_da['CLDTHCK_MISR'] = cldthick_misr_msk # add the new thick cloud variable back in
cldthick_misr_msk = cam4_da[['CLDTHCK_MISR','time_bnds']] # select the new variable and the time_bnds variable

In [32]:
cam4_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam4_runname = 'cam4_1deg_release_amip'

cldthick_misr_msk.to_netcdf('%s/%s/%s.CLDTHCK_MISR.nc' % (cam4_tempdir,cam4_runname,cam4_runname))

#### Manually create the thick-high MODIS cloud files

__Note that the pressure units differ between COSP1.3 and COSP2, so indexing works better than using a where statement (and is also far faster)!__

#### CAM6

In [36]:
# select the three highest x two thickest histogram boxes
cldthick_modis = cam6_da['CLMODIS'][:,-3:,-2:,:,:].sum(dim=['cosp_prs','cosp_tau_modis'],skipna=True)

mask = cam6_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_modis_msk = cldthick_modis.where(mask > -100)

cam6_da['CLDTHCK_MODIS'] = cldthick_modis_msk # add the new thick cloud variable back in
cldthick_modis_msk = cam6_da[['CLDTHCK_MODIS','time_bnds']] # select the new variable and the time_bnds variable

In [39]:
cam6_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam6_runname = 'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1'

cldthick_modis_msk.to_netcdf('%s/%s/%s.CLDTHCK_MODIS.nc' % (cam6_tempdir,cam6_runname,cam6_runname))

#### CAM5

For both COSP versions, the first height index corresponds to a non-retrieval and should be excluded.

In [49]:
# select the three highest x two thickest histogram boxes
cldthick_modis = cam5_da['CLMODIS'][:,-3:,-2:,:,:].sum(dim=['cosp_prs','cosp_tau_modis'],skipna=True)

mask = cam5_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_modis_msk = cldthick_modis.where(mask > -100)

cam5_da['CLDTHCK_MODIS'] = cldthick_modis_msk # add the new thick cloud variable back in
cldthick_modis_msk = cam5_da[['CLDTHCK_MODIS','time_bnds']] # select the new variable and the time_bnds variable

In [50]:
cam5_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam5_runname = 'cam5_1deg_release_amip'

cldthick_modis_msk.to_netcdf('%s/%s/%s.CLDTHCK_MODIS.nc' % (cam5_tempdir,cam5_runname,cam5_runname))

#### CAM4

In [46]:
# select the three highest x two thickest histogram boxes
cldthick_modis = cam4_da['CLMODIS'][:,-3:,-2:,:,:].sum(dim=['cosp_prs','cosp_tau_modis'],skipna=True)

mask = cam4_da['CLD_MISR'][:,0,0,:,:].fillna(-999) # mask where the histogram is nans, easy

cldthick_modis_msk = cldthick_modis.where(mask > -100)

cam4_da['CLDTHCK_MODIS'] = cldthick_modis_msk # add the new thick cloud variable back in
cldthick_modis_msk = cam4_da[['CLDTHCK_MODIS','time_bnds']] # select the new variable and the time_bnds variable

In [47]:
cam4_tempdir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'
cam4_runname = 'cam4_1deg_release_amip'

cldthick_modis_msk.to_netcdf('%s/%s/%s.CLDTHCK_MODIS.nc' % (cam4_tempdir,cam4_runname,cam4_runname))